In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('../data/external/vacinacao-se.csv', sep=';', parse_dates=['paciente_datanascimento', 'vacina_dataaplicacao'])
age_groups = [(0, 17), (18, 35), (36, 59), (60, 200)]

In [ ]:
df = df.assign(vacinacao_completa=(df.vacina_descricao_dose == '2ª Dose').where((df.vacina_fabricante_nome != 'JANSSEN'), True))
df_municipios = df['estabelecimento_municipio_nome'].drop_duplicates().to_frame('municipios')

In [ ]:
dfs = []
for i in range(1, 13):
    date = datetime.strptime('01-' + str(i) + '-2021', '%d-%m-%Y')
    
    df_ages = df_municipios
    for group in age_groups:
        temp1_F = df[   (df['paciente_idade'] > group[0]) &
                        (df['paciente_idade'] < group[1]) &
                        (df['vacina_dataaplicacao'] < date) &
                        (df['vacinacao_completa'] == True) &
                        (df['paciente_enumsexobiologico'] == 'F')].groupby(['estabelecimento_municipio_nome']).size().to_frame('vacinados_parcial_F_' + str(group[1]))
        df_ages = df_ages.merge(temp1_F, how='left', left_on='municipios', right_on='estabelecimento_municipio_nome').fillna(0)

        temp1_M = df[   (df['paciente_idade'] > group[0]) &
                        (df['paciente_idade'] < group[1]) &
                        (df['vacina_dataaplicacao'] < date) &
                        (df['vacinacao_completa'] == False) &
                        (df['paciente_enumsexobiologico'] == 'M')].groupby(['estabelecimento_municipio_nome']).size().to_frame('vacinados_parcial_M_' + str(group[1]))
        df_ages = df_ages.merge(temp1_M, how='left', left_on='municipios', right_on='estabelecimento_municipio_nome').fillna(0)

        temp2_F = df[   (df['paciente_idade'] > group[0]) &
                        (df['paciente_idade'] < group[1]) &
                        (df['vacina_dataaplicacao'] < date) &
                        (df['vacinacao_completa'] == True) &
                        (df['paciente_enumsexobiologico'] == 'F')].groupby(['estabelecimento_municipio_nome']).size().to_frame('vacinados_completo_F_' + str(group[1]))
        df_ages = df_ages.merge(temp2_F, how='left', left_on='municipios', right_on='estabelecimento_municipio_nome').fillna(0)

        temp2_M = df[   (df['paciente_idade'] > group[0]) &
                        (df['paciente_idade'] < group[1]) &
                        (df['vacina_dataaplicacao'] < date) &
                        (df['vacinacao_completa'] == True) &
                        (df['paciente_enumsexobiologico'] == 'M')].groupby(['estabelecimento_municipio_nome']).size().to_frame('vacinados_completo_M_' + str(group[1]))
        df_ages = df_ages.merge(temp2_M, how='left', left_on='municipios', right_on='estabelecimento_municipio_nome').fillna(0)

    df_ages['date'] = date

    dfs.append(df_ages)
final_df = pd.concat(dfs)

In [ ]:
final_df

,municipios,vacinados_parcial_F_17,vacinados_parcial_M_17,vacinados_completo_F_17,vacinados_completo_M_17,vacinados_parcial_F_35,vacinados_parcial_M_35,vacinados_completo_F_35,vacinados_completo_M_35,vacinados_parcial_F_59,vacinados_parcial_M_59,vacinados_completo_F_59,vacinados_completo_M_59,vacinados_parcial_F_200,vacinados_parcial_M_200,vacinados_completo_F_200,vacinados_completo_M_200,date
0,NOSSA SENHORA DO SOCORRO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-01-01
1,LAGARTO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-01-01
2,CANINDE DE SAO FRANCISCO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-01-01
3,MARUIM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-01-01
4,ARACAJU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,FEIRA NOVA,0.0,45.0,0.0,0.0,319.0,573.0,319.0,240.0,628.0,642.0,628.0,485.0,338.0,302.0,338.0,287.0,2021-12-01
71,SAO DOMINGOS,0.0,110.0,0.0,0.0,288.0,903.0,288.0,192.0,670.0,1177.0,670.0,466.0,672.0,643.0,672.0,597.0,2021-12-01
72,TELHA,0.0,95.0,0.0,0.0,212.0,311.0,212.0,165.0,380.0,444.0,380.0,347.0,141.0,158.0,141.0,130.0,2021-12-01
73,SANTO AMARO DAS BROTAS,1.0,1.0,1.0,0.0,37.0,65.0,37.0,24.0,78.0,333.0,78.0,51.0,149.0,460.0,149.0,116.0,2021-12-01


In [ ]:
final_df.to_csv('../data/processed/final_vacinacao.csv', index=False)